# fz-brent: Example with fzd

This notebook demonstrates how to use the Brent root-finding algorithm **coupled with a model** via `fz.fzd()`.

`fzd()` couples an algorithm with a model and calculators to iteratively:
1. Ask the algorithm for points to evaluate
2. Run the model at those points using calculators
3. Feed results back to the algorithm
4. Repeat until the algorithm signals completion

**Requirements:** R, `rpy2`, and a working model plugin (e.g., fz-Model).

## Setup: Install Dependencies

In [ ]:
# Install fz framework with rpy2
%pip install -q git+https://github.com/Funz/fz.git
%pip install -q rpy2

import fz

# Install the Model template plugin (for testing)
fz.install("Model")

# Install this R algorithm plugin
fz.install_algorithm("https://github.com/Funz/fz-brent")

print("Setup complete!")

## Prepare Input Files

Download example input files from the fz-Model repository.

In [ ]:
import urllib.request
import os

# Download example input file
os.makedirs("examples/Model", exist_ok=True)
url = "https://raw.githubusercontent.com/Funz/fz-Model/main/examples/Model/input.txt"
urllib.request.urlretrieve(url, "examples/Model/input.txt")

# Show the input file content
with open("examples/Model/input.txt") as f:
    content = f.read()
print("Input file content:")
print(content)

## Verify Setup

Check that the input file has variables and the algorithm is accessible.

In [ ]:
# Check input variables
variables = fz.fzi("examples/Model/input.txt")
print("Input variables found:")
for var_name, var_info in variables.items():
    print(f"  {var_name}: {var_info}")

# Check that R algorithm is loadable
from fz.algorithms import load_algorithm
algo = load_algorithm("Brent", ytarget=0.0, ytol=0.01, xtol=0.01)
print(f"\nR Algorithm loaded: {type(algo).__name__}")

## Run fzd with Brent Algorithm

Use `fz.fzd()` to couple the Brent algorithm with the model.

The algorithm runs in R (via rpy2), while the model runs via the calculator system.

**Note:** Brent is a 1D root-finding algorithm, so only one input variable should be varied.

In [ ]:
import fz

# Run iterative root finding
analysis = fz.fzd(
    input_path="examples/Model/input.txt",
    input_variables={"x": "[0;10]"},
    model="Model",
    output_expression="result",
    algorithm="Brent",
    algorithm_options={"ytarget": 0.0, "ytol": 0.01, "xtol": 0.01, "max_iterations": 100},
    calculators="localhost_Model",
    analysis_dir="analysis_results_brent"
)

print("\nfzd completed!")
print(type(analysis))

## Explore Results

In [ ]:
# Display the analysis results
if isinstance(analysis, dict):
    if 'text' in analysis:
        print("Analysis text:")
        print(analysis['text'])
    if 'data' in analysis:
        print("\nAnalysis data:")
        for k, v in analysis['data'].items():
            print(f"  {k}: {v}")
else:
    # May be a DataFrame
    print(analysis)

## Passing Algorithm Options

Algorithm options can be passed in several ways:

In [ ]:
# Method 1: As a Python dict
options_dict = {"ytarget": 0.0, "ytol": 0.001, "xtol": 0.001, "max_iterations": 50}
print(f"Method 1 (dict): {options_dict}")

# Method 2: As a JSON string
options_json = '{"ytarget": 0.0, "ytol": 0.001, "xtol": 0.001}'
print(f"Method 2 (JSON string): {options_json}")

# Method 3: As a path to a JSON file
import json
os.makedirs("config", exist_ok=True)
with open("config/algo_options.json", "w") as f:
    json.dump({"ytarget": 0.0, "ytol": 0.001, "xtol": 0.001}, f)
print(f"Method 3 (JSON file): config/algo_options.json")

print("\nAll three methods are equivalent when passed to fzd(..., algorithm_options=...)")